## C卷测试流程

### C卷使用dockerID初步筛选
根据每个赛道各用户在B卷测试中的最高分作为C卷测试dockerID
>输出：`test_list_init.xlsx`

In [7]:
import os
import sys
sys.path.append('../..')
from connector import Connector

import pandas as pd

database = Connector()

init_record = {
    'pid': None,
    'pull': {'status': 'WAITING', 'cost': ''},
    'test': {'status': 'WAITING', 'cost': ''},
    'evaluate': {'status': 'WAITING', 'cost': ''},
    'upload': {'status': 'WAITING', 'cost': ''},
}

def get_competition_submits():
    script = f"SELECT * FROM submit WHERE (paperType='B') AND (competitionId IN ('2023031405', '2023031901', '2023031902', '2023031903'))"
    submits = database._db_fetch(script)
    return submits

def convert_time(time_use):
    time_use = float(time_use)
    h = int(time_use // 3600)
    m = int((time_use - h * 3600) // 60)
    s = int(time_use - h * 3600 - m * 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

def is_timeout(time_use, competitionId):
    time_use = float(time_use)
    if competitionId == '2023031405':
        if time_use > 3600 * 30:
            return True
        else:
            return False
    elif competitionId in ['2023031901', '2023031902', '2023031903']:
        if time_use > 3600 * 10:
            return True
        else:
            return False
    else:
        return False
    
def is_testing(submits):
    for s in submits['status']:
        if s == 'TESTING':
            return True
    return False

if __name__ == "__main__":
    excel_info = {
        'competitionName': [],
        'supervisorName': [],
        'school': [],
        'userName': [],
        'mobile': [],
        'dockerId': [],
        'score': [],
        'submitTime': [],
        'isTesting': [],
        'submitId': [],
    }
    
    submits = get_competition_submits()
    submits_df = pd.DataFrame(submits, columns=database.columns)
    for competition, competition_rows in submits_df.groupby('competitionId'):
        competitionName = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competition)[0][0]
        competition_rows.sort_values(by='score', ascending=True, inplace=True)
        for user, user_rows in competition_rows.groupby('submitterId'):
            user_info = database._get_value(table='user', field='userName, mobile, email, school, supervisorName', logic='AND', userId=user)[0]
            if user_info[0] in ['标杆算法1-IDM', '标杆算法2-Lattice', 'yangyh408']:
                continue
            excel_info['userName'].append(user_info[0])
            excel_info['mobile'].append(user_info[1])
            # excel_info['email'].append(user[2])
            excel_info['school'].append(user_info[3])
            excel_info['supervisorName'].append(user_info[4])
            
            excel_info['competitionName'].append(competitionName)
            
            if is_testing(user_rows):
                excel_info['isTesting'].append('TRUE')
            else:
                excel_info['isTesting'].append(None)
            
            user_rows.sort_values(by='score', ascending=False, inplace=True)
            highest_score = user_rows.iloc[0]
            excel_info['submitId'].append(highest_score['submitId'])
            excel_info['dockerId'].append(highest_score['dockerId'])
            excel_info['score'].append(highest_score['score'])
            excel_info['submitTime'].append(highest_score['submitTime'])
            
    excel_df = pd.DataFrame(excel_info)
    excel_df.to_excel('test_list_init.xlsx', index=False)

### 手动确认C卷测试内容
根据`test_list_init.xlsx`统计选手需求并汇总成`test_list.xlsx`

### 确认C卷测试情况
根据`test_list.xlsx`确定B卷信息登记无误并汇总C卷测试情况
> 输出：排行榜文件`leaderboard.xlsx`

In [110]:
import pandas as pd
import os
import sys
sys.path.append('../..')
from connector import Connector
import warnings
warnings.filterwarnings('ignore')

database = Connector()

df = pd.read_excel('./test_list.xlsx')
tests = df[['competitionName', '团队老师', '主用户名', '最终确认DockerID', 'B卷得分']].dropna(subset=['最终确认DockerID'])
# tests = df[['competitionName', '团队老师', '主用户名', '最终确认DockerID', 'B卷得分']].dropna()

C_competition_map = {
    '2023031405': '2023052701',
    '2023031901': '2023052702',
    '2023031902': '2023052703',
    '2023031903': '2023052704',
}
B_submitIds = []
C_scores = []
C_statuses = []
C_submitIds = []
for index, row in tests.iterrows():
    competitionId = database._get_value(table='competition', field='competitionId', logic='AND', competitionName=row['competitionName'])[0][0]
    userId = database._get_value(table='user', field='userId', logic='AND', userName=row['主用户名'])[0][0]
    try:
        script = f"SELECT score, submitId FROM submit WHERE competitionId='{competitionId}' AND submitterId='{userId}' AND dockerId='{row['最终确认DockerID'].strip()}' ORDER BY score DESC LIMIT 1"
        B_score, submitId = database._db_fetch(script)[0]
        if row['B卷得分'] != B_score:
            print(competitionId, userId, row['最终确认DockerID'], row['B卷得分'], B_score)
        B_submitIds.append(submitId)
    except:
        print(competitionId, userId, row['主用户名'], row['最终确认DockerID'])
    
    try:
        script = f"SELECT status, score, submitId FROM submit WHERE competitionId='{C_competition_map[competitionId]}' AND dockerId='{row['最终确认DockerID'].strip()}' ORDER BY score DESC LIMIT 1"
        C_stauts, C_score, submitId = database._db_fetch(script)[0]
        C_statuses.append(C_stauts)
        C_submitIds.append(submitId)
        if not C_score and  C_stauts not in ['QUEUING', 'WAITING', 'TESTING']:
            C_scores.append(0)
        else:
            C_scores.append(C_score)
    except:
        C_scores.append(None)
        C_submitIds.append(None)
        C_statuses.append('未测试')
    
tests['B卷排名'] = tests.groupby('competitionName')['B卷得分'].rank(method='min', ascending=False)
tests['B卷submitId'] = B_submitIds
tests['C卷测试状态'] = C_statuses
tests['C卷得分'] = C_scores
tests['C卷排名'] = tests.groupby('competitionName')['C卷得分'].rank(method='min', ascending=False)
tests['C卷submitId'] = C_submitIds

tests['总得分'] = tests['B卷得分'] * 0.7 + tests['C卷得分'] * 0.3
tests['总排名'] = tests.groupby('competitionName')['总得分'].rank(method='min', ascending=False)

tests.sort_values(by=['competitionName', '总得分'], ascending=False, inplace=True)

In [111]:
# 创建一个ExcelWriter对象，并将每个分组的数据写入到不同的Excel工作表中
writer = pd.ExcelWriter('leaderboard.xlsx')
for name, group in tests.groupby('competitionName'):
    group.assign()
    group.to_excel(writer, sheet_name=str(name), index=False)
    
# 保存并关闭ExcelWriter对象
writer.close()

### C卷场景筛选
> 输入：`C_select_scenes.xlsx`

In [18]:
import os
import urllib.request
import zipfile
import pandas as pd
import shutil
from tqdm import tqdm
import sys
sys.path.append('../..')
from connector import Connector
from uploader import Uploader

database = Connector()
base_dir = "/home/ubuntu/onsite-test-server/temp"

def download_result(submitId, url):
    filename = url.split('/')[-1]
    filepath = os.path.join(base_dir, filename)
    extract_dir = os.path.join(base_dir, submitId)
    if os.path.exists(extract_dir):
        shutil.rmtree(extract_dir)
    if not os.path.exists(filepath):
        urllib.request.urlretrieve(url, filepath)
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    os.remove(filepath)
    
def filter_files(submitId, new_submitId, select_scene):
    submit_dir = os.path.join(base_dir, submitId)
    new_submit_dir = os.path.join(base_dir, new_submitId)
    score_path = os.path.join(submit_dir, 'score.csv')
    
    columns = ['Scenario', 'Safety', 'Efficiency', 'Comfort', 'Total']
    
    if os.path.exists(score_path):
        scores = pd.read_csv(score_path)[columns]
        select_scores = pd.DataFrame(columns=columns)
        for scene in select_scene:
            row = scores[scores['Scenario'] == scene]
            if not row.empty:
                select_scores = pd.concat([select_scores, row], ignore_index=True)
            else:
                select_scores = pd.concat([select_scores, pd.DataFrame([[scene,0,0,0,0]], columns=columns)], ignore_index=True)
        mean_row = select_scores[columns[1:]].mean(axis=0).tolist()
        mean_row.insert(0, 'Mean')
        select_scores = pd.concat([select_scores, pd.DataFrame([mean_row], columns=columns)], ignore_index=True)
        select_scores.round(4).to_csv(score_path, index=False)
        
    for file_name in os.listdir(submit_dir):
        if file_name.endswith('.csv') and not file_name.startswith('score'):
            if file_name[:-11] not in select_scene:
                os.remove(os.path.join(submit_dir, file_name))
                
    os.rename(submit_dir, new_submit_dir)

def generate_C_submition(submitId, userName):
    C_competition_map = {
        '2023052701': '2023031405',
        '2023052702': '2023031901',
        '2023052703': '2023031902',
        '2023052704': '2023031903',
    }
    res = database._get_value(table='submit', field='*', logic='AND', submitId=submitId)
    res = dict(zip(database.columns, res[0]))
    res['competitionId'] = C_competition_map[res['competitionId']]
    
    submitterId = database._get_value(table='user', field='userId', logic='AND', userName=userName)[0][0]
    res['submitterId'] = submitterId
    res['submitId'] = res['submitId'][:17] + submitterId
    
    res['paperType'] = 'C'
    res['resultLink'] = None
    return res

select_scenes = {}
select_scenes_df = pd.read_excel('C_select_scenes.xlsx')
for competitionName, scenes in select_scenes_df.items():
    select_scenes[competitionName] = scenes.dropna().tolist()

leaderboard = pd.read_excel('leaderboard.xlsx', sheet_name=None)
for competition, rows in leaderboard.items():
    with tqdm(total=len(rows)) as _tqdm:
        _tqdm.set_description(f"{competition}")    
        for index, row in rows.iterrows():
            _tqdm.update(1)
            resultLink = database._get_value(table='submit', field='resultLink', logic='AND', submitId=row['C卷submitId'])[0][0]
            if resultLink:
                new_submit_info = generate_C_submition(row['C卷submitId'], row['主用户名'])
                download_result(row['C卷submitId'], resultLink)
                filter_files(row['C卷submitId'], new_submit_info['submitId'], select_scenes[competition])
                uploader = Uploader(base_dir, new_submit_info)
                try:
                    uploader._upload_file()
                    new_submit_info['resultLink'] = uploader.result_link
                except Exception as e:
                    print(f"UPLOAD ERROR: {row['主用户名']} ({row['C卷submitId']}->{new_submit_info['submitId']})")
                # database.insert(table = 'submit', info = new_submit_info)
                print(new_submit_info)
            else:
                print(f"{row['C卷submitId']}未找到下载链接!")
            break
        break

第一届OnSite自动驾驶算法挑战赛（交叉口专项赛）:   6%|▌         | 1/17 [00:02<00:34,  2.15s/it]

{'submitId': 's_20230601103948_20230417021453', 'submitterId': '20230417021453', 'competitionId': '2023031903', 'submitTime': datetime.datetime(2023, 6, 1, 10, 39, 48), 'testTime': datetime.datetime(2023, 6, 1, 10, 39, 52), 'score': 77.2382, 'dockerId': '1982777/buaatrans2023:6.0.0', 'paperType': 'C', 'resultLink': 'https://resource.onsite.run/results/2023031903/20230417021453&2023-06-01_10-39-48.zip', 'status': 'SUCCESS', 'testDetail': '{"pid": 3972055, "pull": {"cost": "1.97", "status": "SUCCESS"}, "test": {"cost": "59993.45", "status": "SUCCESS"}, "upload": {"cost": "5.21", "status": "SUCCESS"}, "evaluate": {"cost": "1836.55", "status": "SUCCESS"}}'}


In [15]:
import sys
sys.path.append('../..')
from connector import Connector
from uploader import Uploader

database = Connector()

def generate_C_submition(submitId, userName):
    C_competition_map = {
        '2023052701': '2023031405',
        '2023052702': '2023031901',
        '2023052703': '2023031902',
        '2023052704': '2023031903',
    }
    res = database._get_value(table='submit', field='*', logic='AND', submitId=submitId)
    res = dict(zip(database.columns, res[0]))
    res['competitionId'] = C_competition_map[res['competitionId']]
    
    submitterId = database._get_value(table='user', field='userId', logic='AND', userName=userName)[0][0]
    res['submitterId'] = submitterId
    res['submitId'] = res['submitId'][:17] + submitterId
    
    res['paperType'] = 'C'
    res['resultLink'] = None
    return res

base_dir = "/home/ubuntu/onsite-test-server/temp"
submit_info = generate_C_submition('s_20230601103948_20230316121333', 'BUAATRANS2023')
uploader = Uploader(base_dir, submit_info)
try:
    uploader._upload_file()
    submit_info['resultLink'] = uploader.result_link
except Exception as e:
    print(f"UPLOAD ERROR: {submit_info['submitId']}")

https://resource.onsite.run/results/2023031903/20230417021453&2023-06-01_10-39-48.zip


In [44]:
import os
import urllib.request
import zipfile
import shutil

def download_result(submitId, url):
    base_dir = "/home/ubuntu/onsite-test-server/temp"
    filename = url.split('/')[-1]
    filepath = os.path.join(base_dir, filename)
    extract_dir = os.path.join(base_dir, submitId)
    if os.path.exists(extract_dir):
        shutil.rmtree(extract_dir)
    if not os.path.exists(filepath):
        urllib.request.urlretrieve(url, filepath)
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    os.remove(filepath)

download_result('s_20230601103948_20230316121333', 'https://resource.onsite.run/results/2023052704/20230316121333&2023-06-01_10-39-48.zip')

In [108]:
def filter_files(submitId, new_submitId, select_scene):
    base_dir = "/home/ubuntu/onsite-test-server/temp"
    submit_dir = os.path.join(base_dir, submitId)
    new_submit_dir = os.path.join(base_dir, new_submitId)
    score_path = os.path.join(submit_dir, 'score.csv')
    
    columns = ['Scenario', 'Safety', 'Efficiency', 'Comfort', 'Total']
    
    if os.path.exists(score_path):
        scores = pd.read_csv(score_path)[columns]
        select_scores = pd.DataFrame(columns=columns)
        for scene in select_scene:
            row = scores[scores['Scenario'] == scene]
            if not row.empty:
                select_scores = pd.concat([select_scores, row], ignore_index=True)
            else:
                select_scores = pd.concat([select_scores, pd.DataFrame([[scene,0,0,0,0]], columns=columns)], ignore_index=True)
        mean_row = select_scores[columns[1:]].mean(axis=0).tolist()
        mean_row.insert(0, 'Mean')
        select_scores = pd.concat([select_scores, pd.DataFrame([mean_row], columns=columns)], ignore_index=True)
        select_scores.round(2).to_csv(score_path, index=False)
        
    for file_name in os.listdir(submit_dir):
        if file_name.endswith('.csv') and not file_name.startswith('score'):
            if file_name[:-11] not in select_scene:
                os.remove(os.path.join(submit_dir, file_name))
                
    os.rename(submit_dir, new_submit_dir)
        
filter_files('s_20230601103948_20230316121333', select_scenes[competition])